In [ ]:
import pandas as pd
import re

In [ ]:
#install tensorflow 2.8
!pip install tensorflow==2.8.0
#vsc pakai tanda %


In [ ]:
#ambil link project dataset
url = 'https://raw.githubusercontent.com/RashidMaulana/Anticede/main/Machine%20Learning/data_2/data%20-%20datatest_2.csv'
dataset = pd.read_csv(url)
dataset.head()

In [ ]:
dataset.tail()

In [ ]:
# Change text to lowercase
def proses_data(text):
    newText = text.lower()
    return newText
#ini case kalau semua mau ngerun semua data Komputerku kentang maka ngefreeze)
# dataset['comment_text'].apply(lambda x: print(x))
#ini case kalau semua mau ngerun 5 data pertama
testData = dataset['original_text']
testData = testData.apply(lambda x: proses_data(x))

#menghilangkan link dan tag html
import re
def link_remover(text):
    url_regex = re.compile(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?')
    return url_regex.sub('', text)
testData = testData.apply(lambda x: link_remover(x))

#menghilangkan kalimat bersama tag
import re
def tag_remover(text):
    tag_regex = re.compile(r'<[^>]+>')
    return tag_regex.sub('', text)
testData = testData.apply(lambda x: tag_remover(x))

# menghilangkan tanda baca dan karakter spesial
import re
def karakter_spesial(text):
    newText = re.sub('[^a-zA-Z0-9\s]', '', text)
    return newText
testData = testData.apply(lambda x: karakter_spesial(x))

# menghilangkan spasi berlebihan
def Spasi_berlebihan(text):  
    newText = re.sub('\s', ' ', text)
    return newText
testData = testData.apply(lambda x: Spasi_berlebihan(x))

# menghilangkan angka berlebih
def angka_berlebihan(text):
    newText = re.sub('[0-9]', '', text)
    return newText
testData = testData.apply(lambda x: angka_berlebihan(x))

In [ ]:
dataset['pornografi'].value_counts()

In [ ]:
dataset['sara'].value_counts()

In [ ]:
dataset['radikalisme'].value_counts()

In [ ]:
dataset['pencemaran_nama_baik'].value_counts()

In [ ]:
#oov tokenizer
tokenizer = Tokenizer(num_words = 100, oov_token="<OOV>")
tokenizer.fit_on_texts(testData)
word_index = tokenizer.word_index #menampilkan tabel kata yang ada di dataset
print(f'number of words in word_index: {len(word_index)}')
# Print the word index
print(f'word_index: {word_index}')
print()
# Generate and pad the sequences
sequences = tokenizer.texts_to_sequences(testData)
padded = pad_sequences(sequences, padding='post')
# Print a sample headline
index = 2
print(f'sample headline: {testData[index]}')
print(f'padded sequence: {padded[index]}')
print()

In [ ]:
# menampilkannya dalam bentuk integer
sequences = tokenizer.texts_to_sequences(testData)
print(sequences)

In [ ]:
#melatih data train dengan tensorflow
url_train_data = 'https://raw.githubusercontent.com/RashidMaulana/Anticede/main/Machine%20Learning/data_2/data%20-%20datatrain_2.csv'
train_data = pd.read_csv(url_train_data)

# Change text to lowercase
def proses_data(text):
    newText = text.lower()
    return newText
traindata = train_data['original_text']
traindata = traindata.apply(lambda x: proses_data(x))

#menghilangkan link dan tag html
def link_remover(text):
    url_regex = re.compile(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?')
    return url_regex.sub('', text)
traindata = traindata.apply(lambda x: link_remover(x))

#menghilangkan kalimat bersama @name
def name_remover(text):
    name_regex = re.compile(r'@[\w_-]+')
    return name_regex.sub(' ', text)
traindata = traindata.apply(lambda x: name_remover(x))

#menghilangkan tanda baca dan karakter spesial
def karakter_spesial(text):
    newText = re.sub('[^a-zA-Z0-9\s]', ' ', text)
    return newText
traindata = traindata.apply(lambda x: karakter_spesial(x))

#menghilang angka yang nempel dengan kalimat
def angka_berlebihan(text):
    newText = re.sub('[0-9]', '', text)
    return newText
traindata = traindata.apply(lambda x: angka_berlebihan(x))

#menghilangkan spasi berlebihan
def Spasi_berlebihan(text):  
    newText = re.sub('\s+', '', text)
    newText = re.sub('^\s+', '', text)
    return newText
traindata = traindata.apply(lambda x: Spasi_berlebihan(x))
traindata.head()


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
#initialize the tokenizer class
tokenizer = Tokenizer()
tokenizer.fit_on_texts(traindata)
word_index = tokenizer.word_index
print(f'number of words in word_index: {len(word_index)}')
print(f'word_index: {word_index}')

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
#initialize the sequences list
sequences = tokenizer.texts_to_sequences(traindata)
padded = pad_sequences(sequences, padding='post')
print(f'shape of padded sequences: {padded.shape}')
label = train_data['pornografi']
label.head()
#print the sample headline
index = 2
print(f'sample headline: {traindata[index]}')
inputs = padded[:-1]
labels = padded[1:]
# Print the shape of inputs and labels
print(f'shape of inputs: {inputs.shape}')

In [ ]:
#Hyperparameters
Embedding_Dim = 100
lstm = 128
dense = 64
#build the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
model = Sequential()
model.add(Embedding(len(word_index) + 1, 100, input_length=padded.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
##sepertinya aku salah di model trainingnya coba kamu ubah2